In [ ]:
import pandas as pd
import unicodedata
from tqdm.auto import tqdm, trange
import numpy as np
import spacy
from dictionary_lookup import Dictionary
import random
import sacrebleu
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import re
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup
pd.options.mode.chained_assignment = None 
nlp = spacy.load('ro_core_news_sm')
random.seed(42)

##### Here we create a dataset that consists of the text splitted at sentence or subsentence level (it's actually splitted by most commmon end of sentence characters: .,?,!). 

In [2]:
df = pd.read_csv("../dataset/updated_parallel_corpus.csv", nrows=1000)
df_ro_rup = df[['ro', 'rup']]
df_ro_rup.dropna(inplace=True)

# The text is splitted into sentences so we can construct a dataset with more granular data
def split_text(text):
    sentence_endings = re.compile(r'(?<=[.!?])\s+')
    sentences = sentence_endings.split(text)
    return sentences

df_ro_rup.loc[:, 'ro_sentences'] = df_ro_rup.loc[:, 'ro'].apply(split_text)
df_ro_rup.loc[:, 'rup_sentences'] = df_ro_rup.loc[:, 'rup'].apply(split_text)
df_ro_rup.drop(columns=['ro', 'rup'], inplace=True)

# There are some rows that have different number of sentences in the two columns, we will join them
# The output will be a list of strings, in case of joining it will be a list with one string
mask = df_ro_rup['ro_sentences'].apply(len) != df_ro_rup['rup_sentences'].map(len)
df_ro_rup[mask] = df_ro_rup[mask].map(lambda x: [' '.join(x)])

# Now we will create a dataset with one sentence per row
df_ro_rup_data = pd.DataFrame(columns=['ro', 'rup'])
for index, row in df_ro_rup.iterrows():
    for ro_sentence, rup_sentence in zip(row['ro_sentences'], row['rup_sentences']):
        df_ro_rup_data = pd.concat([df_ro_rup_data, pd.DataFrame({'ro': [ro_sentence], 'rup': [rup_sentence]})], ignore_index=True)



#### Adding words from the dictionary to the dataset

In [3]:
file_path = '../others/Papahagi.xls'
dict = Dictionary(file_path, column_names=['pos', 'aromanian', 'romanian', 'source_lang', 'details', 'forms'], unused_columns=['source_lang', 'details', 'forms'])
dict.create_dictionary(original_csv_file="../others/Papahagi_original.csv", transformed_csv_file="../others/Papahagi.csv")
for key in dict.dict.keys():
    for el in dict.dict[key]:
        rup = el[0]
        ro = el[1]
        df_ro_rup_data.loc[len(df_ro_rup_data) + 1] = [ro, rup]
df_ro_rup_data.drop_duplicates(inplace=True)
df_ro_rup_data.reset_index(drop=True, inplace=True)
df_ro_rup_data.to_csv("../dataset/nllb_corpus.csv", index=False)

In [4]:
df = pd.read_csv("../dataset/nllb_corpus.csv")
# Reapply the transformations
df_transformed = df.map(lambda x: ''.join([c for c in unicodedata.normalize('NFKD', x)  if unicodedata.category(c) != 'Mn']) if type(x) == str else x)
# There are some words that have a -mi at the end, we will eliminate them also
df_transformed.replace(r'\s*-\s*mi\b', '', regex=True, inplace=True)
df_transformed.replace(r'\(i\)', 'i', regex=True, inplace=True)
df_transformed.replace('γ', 'y', regex=True, inplace=True)
df_transformed.replace(r'’', '', regex=True, inplace=True)
df_transformed.replace(r'“', '', regex=True, inplace=True)
df_transformed.replace(r'„', '', regex=True, inplace=True)
df_transformed.replace(r' - fig', ' ', regex=True, inplace=True)
df_transformed.replace('"', '', regex=False, inplace=True)

# Some words from the dictionary have a comma with space and a letter at the end, we will eliminate them
df_transformed.replace(r',\s*[a-zA-Z]$', '', regex=True, inplace=True)


df_transformed.columns = [str(q).strip() for q in df_transformed.columns]
df_ro_rup = df_transformed[['ro', 'rup']]
df_ro_rup = df_ro_rup.sample(frac=1).reset_index(drop=True)
df_ro_rup.dropna(inplace=True)
# Split the data into train and test
train_size = int(0.9 * df_ro_rup.shape[0])
df_ro_rup_train = df_ro_rup.iloc[:train_size]
df_ro_rup_test = df_ro_rup.iloc[train_size:]
df_ro_rup_train.reset_index(drop=True, inplace=True)
df_ro_rup_test.reset_index(drop=True, inplace=True)

df_ro_rup.to_csv("../dataset/nllb_corpus.csv", index=False)

df_ro_rup_train.to_csv("../dataset/nllb_corpus_train.csv", index=False)
df_ro_rup_test.to_csv("../dataset/nllb_corpus_test.csv", index=False)


In [5]:
# remove unused variables
del df
del df_ro_rup
del df_transformed
del df_ro_rup_data
del mask
del row

Parts of the following code are inspired by: https://cointegrated.medium.com/how-to-fine-tune-a-nllb-200-model-for-translating-a-new-language-a37fc706b865

In [6]:
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True, src_lang="ro_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer.src_lang = "ron"
inputs = tokenizer(text="Acsite, amiraroanea s-nairi si nu vru", return_tensors="pt")
translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.encode("ro_Latn")[0]
)
print(tokenizer.decode(translated_tokens[0], skip_special_tokens=True))

c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ronanea s-nairi si nu vru


In [7]:

model.cuda();
optimizer = Adafactor(
    [p for p in model.parameters() if p.requires_grad],
    scale_parameter=False,
    relative_step=False,
    lr=1e-4,
    clip_threshold=1.0,
    weight_decay=1e-3,
)
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=1000)

In [8]:
LANGS = [('rup', 'aromanian'), ('ro', 'romanian')]

def get_batch_pairs(batch_size, data=df_ro_rup_train):
    (l1, long1), (l2, long2) = random.sample(LANGS, 2)
    xx, yy = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        xx.append(item[l1])
        yy.append(item[l2])
    return xx, yy, long1, long2

print(get_batch_pairs(15))

(['vintosu', 'mirudhyeaua', 'una -oara', 'ortu', 'znjiura', 'giunami', 'tuv(a)leta', 'cildisescu', 'orthodhoxu', 'lirosu', 'ayunjisescu', 'ciudescu', 'aleapidu', 'Cara lu-arcara, di mica-mica flurii, di nu avea iu s-li ma baga, cindu vinira s-veada s-easte ca le-arca petalile, nidisclisa usa ghine, cucotlu se-astrapse nafoara di use. Tradze-ndreptu la auslu si-i dzite:', 'ni-minari'], ['vantului expus', 'miros', 'candva', 'sfert de piastru', 'zmeura', 'viteji multi', 'marama', 'ului', 'ortodox', 'jegos', 'grabi', 'extazia', 'arunca', 'Il azvarlira in hazna si inghiti cocosul la galbeni, de nu mai avea loc in pipota. Cand venira oamenii craiului sa vada daca si-a dat duhul, numai ce deschisera ei usa ca o si zbughi afara. Merge - glont, la mos si-i zice:', 'nemiscare'], 'aromanian', 'romanian')


In [9]:
batch_size = 16  # 32 already doesn't fit well to 15GB of GPU memory
max_length = 128  
training_steps = 60000  
losses = []  
MODEL_SAVE_PATH = 'test_train'  

In [ ]:
model.train()
x, y, loss = None, None, None

tq = trange(len(losses), training_steps)
for i in tq:
    xx, yy, lang1, lang2 = get_batch_pairs(batch_size)
    try:
        tokenizer.src_lang = lang1
        x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        tokenizer.src_lang = lang2
        y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100

        loss = model(**x, labels=y.input_ids).loss
        loss.backward()
        losses.append(loss.item())

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()

    except RuntimeError as e:  
        optimizer.zero_grad(set_to_none=True)
        x, y, loss = None, None, None
        print('error', max(len(s) for s in xx + yy), e)
        continue

    if i % 1000 == 0:
        print(i, np.mean(losses[-1000:]))

    if i % 1000 == 0 and i > 0:
        model.save_pretrained(MODEL_SAVE_PATH)
        tokenizer.save_pretrained(MODEL_SAVE_PATH)

In [7]:

model = AutoModelForSeq2SeqLM.from_pretrained('test_train')
tokenizer = AutoTokenizer.from_pretrained('test_train')
# load test data
df_ro_rup_test = pd.read_csv("../dataset/nllb_corpus_test.csv")

# Put the on the GPU
model.cuda();
model.eval();

def translate(
    text, src_lang='ro', tgt_lang='rup', 
    a=32, b=3, max_input_length=1024, num_beams=4, **kwargs
):
    """Turn a text or a list of texts into a list of translations"""
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(
        text, return_tensors='pt', padding=True, truncation=True, 
        max_length=max_input_length
    )
    model.eval() # turn off training mode
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams, **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

# Example usage:
t = 'Ma, a lor la si paru c-amintara na vasilie-ntreaga'
print(translate(t, 'aromanian', 'romanian'))

['Dar, a lor i se parea ca-au mentit o vasilie-ntreaga']


In [8]:
df_ro_rup_test['ro_pred'] = ''
df_ro_rup_test['rup_pred'] = ''
# Loop through the DataFrame in chunks of 16
test_len = len(df_ro_rup_test)
for i in tqdm(range(0, test_len, 16)):
    end_index = min(i + 16, test_len)  # Ensure not to exceed the DataFrame length
    
    # Extract the chunk of data
    ro_texts = df_ro_rup_test.loc[i:end_index-1, 'ro'].tolist()
    ls = [len(x) for x in ro_texts]
    if max(ls) > 1000:
        continue
    rup_texts = df_ro_rup_test.loc[i:end_index-1, 'rup'].tolist()
    print(ro_texts)
    # Only perform translation if the input list is not empty
    if ro_texts:
        # Translate works in the following way: I have texts in romanian(ro_texts), I want to translate them to aromanian (aromanian <- romanian)
        df_ro_rup_test.loc[i:end_index-1, 'rup_pred'] = translate(ro_texts, 'aromanian', 'romanian')
    
    if rup_texts:
        df_ro_rup_test.loc[i:end_index-1, 'ro_pred'] = translate(rup_texts, 'romanian', 'aromanian')


# Calculate the BLEU score
bleu_calc = sacrebleu.BLEU(max_ngram_order=2, effective_order=False)
chrf_calc = sacrebleu.CHRF(word_order=2)
print(bleu_calc.corpus_score(df_ro_rup_test['ro'].to_list(), [df_ro_rup_test['ro_pred'].to_list()]))
print(chrf_calc.corpus_score(df_ro_rup_test['ro'].to_list(), [df_ro_rup_test['ro_pred'].to_list()]))
print(bleu_calc.corpus_score(df_ro_rup_test['rup'].to_list(), [df_ro_rup_test['rup_pred'].to_list()]))
print(chrf_calc.corpus_score(df_ro_rup_test['rup'].to_list(), [df_ro_rup_test['rup_pred'].to_list()]))

  0%|          | 0/188 [00:00<?, ?it/s]

['inchisoare', 'covatica', 'fecioara - fata', 'junica', 'paraliza', 'cimpoi', 'discurs', 'sigila', 'ruinat', 'barca', 'boia', 'femeie brava', 'epitaf', 'streasina', 'tuna', 'departe']


  1%|          | 1/188 [00:00<03:03,  1.02it/s]

['Cand soarele da sa apuna, zari sub un copac doi bivoli pe care cineva tocmai ii frigea in doua frigari cat toate zilele.', 'greata', 'repezi', 'Cat traim - multe auzim si multe vedem, dar cu toate astea - nimic nu vom sti ca lumea pana nu punem mana sa si facem ceea ce ne pare ca stim.', 'narghilea', 'flota', 'incovriga', 'burta', 'ticalos', 'curatenie a face', 'Fa-ma pasare cu cioc,', 'praz, mancare de', 'dihor', 'sindrila', 'intens', 'intalni']


  1%|          | 2/188 [00:03<05:14,  1.69s/it]

...

['naut', 'nazuri', 'treia zi', 'izma', 'spargere', 'prea bine-', 'clopotel', 'ingriji', 'pojar', 'bogat', 'taragana', 'inserare', 'netuns', 'imbraca', 'Mahnit adanc, porni spre casa, rusinat de neizbanda lui, de parca se ducea la taierea capului.', 'excrement']


 99%|█████████▉| 187/188 [07:27<00:03,  3.39s/it]

['innopta', 'voal', '- Astuia-i zisei Jumatica, fiindca a fost fata de-asta data.', 'netot', 'pregatit', 'monoton ', 'samur', 'podoaba', 'capaci', 'invesmanta', 'acum', 'nenorocit']


100%|██████████| 188/188 [07:29<00:00,  2.39s/it]


BLEU = 11.77 17.8/7.8 (BP = 1.000 ratio = 1.203 hyp_len = 7507 ref_len = 6242)
chrF2++ = 20.36
BLEU = 12.14 17.4/9.0 (BP = 0.968 ratio = 0.969 hyp_len = 5931 ref_len = 6121)
chrF2++ = 19.81


In [23]:
model.save_pretrained("nllb_aromanian_splitted")
tokenizer.save_pretrained("nllb_aromanian_splitted")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


('nllb_aromanian_splitted\\tokenizer_config.json',
 'nllb_aromanian_splitted\\special_tokens_map.json',
 'nllb_aromanian_splitted\\tokenizer.json')

In [9]:
bleu_calc = sacrebleu.BLEU()
chrf_calc = sacrebleu.CHRF()
print(bleu_calc.corpus_score(df_ro_rup_test['ro'].to_list(), [df_ro_rup_test['ro_pred'].to_list()]))
print(chrf_calc.corpus_score(df_ro_rup_test['ro'].to_list(), [df_ro_rup_test['ro_pred'].to_list()]))
print(bleu_calc.corpus_score(df_ro_rup_test['rup'].to_list(), [df_ro_rup_test['rup_pred'].to_list()]))
print(chrf_calc.corpus_score(df_ro_rup_test['rup'].to_list(), [df_ro_rup_test['rup_pred'].to_list()]))

BLEU = 5.62 17.8/7.8/3.7/1.9 (BP = 1.000 ratio = 1.203 hyp_len = 7507 ref_len = 6242)
chrF2 = 21.91
BLEU = 5.18 17.4/9.0/3.6/1.4 (BP = 0.968 ratio = 0.969 hyp_len = 5931 ref_len = 6121)
chrF2 = 22.08


In [10]:
t1 = ['Grira tu oara laie, ca - na!, iu la s-feate un nic, si-atel isi fu feata-vobira! Ma, a lor la si paru c-amintara na vasilie-ntreaga, - canda la se-aminta Stamaria-n casa! Te s-dzit? - l-aridea inima, ca terlu cindu-i soarle analtu!', '- Dati la oarfan, dati!', 'Una dzua - nu-are lucru - acata si-u-ntreaba ma-sa:', 'Doli ficiori aesti, cari criscura totna deadun, s-loara ahinta vreare, ca bana s-dadea un tr-alantu.', 'E! Nu u da - cara!', 'S-featira fartat treili: hililu-de-avinator, Mica-multu si thiriulu Alumta-caliuri. S-pale li-u deadira-nclo! Ghine ma, Alumta-caliuri nu eara invitat s-imna preayalea, ca virnu satut; ili lo ingricica doli sot si acata s-imna - imnatic de-a lui, cum isi stia. Tu una dzua - featira cale de-un an!', 'Vrura sotli s-li-u suta mintea, s-nu intra tu Padurea-Laie, ca va-si cheara bana; el, prit una ureacle li-intra, prit alanta li-isea.', 'Se-aleapida vulturlu tu nior, si-si paste oclili divarliga. Diparte, diparte - largu multu, andzareaste un puiliu cu un pucic in cioc. S-aurneaste pri nis, lu-acata, s-li-ul lea puciclu, implin cu apa-yie, athanata, si u adute la ficior.', 'Sant indoaua inveturi, cari si aminta masi cu ghivasirea, i cu avdzarea. Sant si indoaua alte lucri, cari nu intra tu caplu a omlui pina s-nu li videm s-cu ocli. Cat banam - multe avdzam si multe videm, ma cu tute aiste - tiva nu stim, pina s-nu bagam mana ca s-adram atea ti na si pare ca stim. Cu alte zboara, voi ca s-dzic eu - ca teoria easte buna, di multe ori, ama cate unoara armane multu ma-nghios di practica.', 'Un carvanar il baga tu ocliu si vru s-ul lea:', 'Museata-loclui, tu fuga, s-toarna si-li cafta a ma-sai s-li da thiriulu si apa-yie, fara moarte. Ma-sa, cu tut ca tinea la lucrile aeste ahardzite, - s-nu-li fringa inima, - li li deade.', '- Ghine, gri mulearea, du-te si lea-u hile-mea, ma - ia ti easte luyuria: la mine suntu patrudzat di feate. Tute museate, si tute s-u aduc una cu alanta. Museata-loclui easte cu nise. S-putearii s-u cunosti - a ta s-hiba!', 'Prota intribare te-li feate ali-nveaste, cind u vidzu, fu:', '- Maca pot tine ahintu lisor s-fat un ahtare lucru, pina mine s-ni adari na cale astirnuta masi cu asime, di la palatea mea pina la a ta. S-puteari s-u fat aesta, va-t dau giumitat di amirarile si hile-mea inveasta.', '- Ti numa-li bagasi, Chira-Mara?', 'Ma vidzu ca nu s-mina virnu, tut s-cama apruchea dhicunarlu. Aproachi-te, aproachi-te, li-u deade nurizma, di pita tu nari, si cindu si-u baga dininte, da-li, s-da-li - prota peturlu di pri-supra, apoia veardzile, calcinile, peturlu di pri-ghios - nu-si alasa tiva.']
t2 = ['Di un, de-alantu, vine la ciurlu-papa sa-ncarca yiplu. Ghine ma, preftu - tut cinticlu te si stea:', 'Pri aestu munte, virnu nu putea sa se-alina. Cum s-faca si cum s-adara emburlu, s-lu-alina ficiorlu? Acata di-l coase tu na cheale, s-lu-arunca tu cuprii. Aoa s-adunara corghili si ornili, s-tradze un, tradze-alantu, il mutara pri muntile cu yi            zi. Ficiorlu, dupa te arupsira corghili chealea, isi dit ea si s-baga s-aduna yiamandzi. Atumtea ili gri emburlu:', 'Prota intribare te-li feate ali-nveaste, cind u vidzu, fu:', 'Cindu agiumse maratlu gione-n hoara, li se-arupse inima di te vidzu: casile tute - ponde s-erme! Canda tricu pusclea pre-aclo: ni cipit di om si di pravda nu se-avdza! Ili vine a lui ergu, ca vombira va le-adra tute aeste laet.', '- Barbate - ili dzitea mulearea - alasa-te de-avinare s-mutrea de-acata virnu altu lucru! Ca, nu vedz pri ti cale agiumsim?', 'Dupa te mutri tu toate partile si nu putu s-lu andzareasca, ili yinea s-creapa di foc s-di arau, cum s-doarma si-li ascapa pulilu.', 'Tradze ficiorlu ninga una, s-ninga una, s-ninga una - pina li guli tute fusechile, si lamnea, cu tuta goada ti-li fatea, nu cadea!', 'Armase vasilia fara domnu. Atumtea ficiorlu - ca atel cama gionile ti se-afla, s-alina pri scamnu si s-feate nis amira, iar Museata-loclui, amiroane, - s-binara na bana di domni!', 'Apoia intra la cinile mare. Cu pirgaclu a neu il dipuse si pe-aistu si lo citi paradz di amalama putu s-poarta. Inclise udaia museat, si cindu s-iasa, s-aduse aminte ca agirsi minarlu! Mina barnul, tra s-iasa nafoara, si cindu agiundue pi budza di gura, lu intreaba moasa:', '- Cinta s-ni dai mulile cit cama ayonea, s-ni dai s-yiptul, s-tute! Acsi na fu acatatura s-ligatura noastra, si - maca te ariseaste, da-te cu d-alaga s-te-acat s-cu altu la minciuni...', '- E, ti numa-li bagasi s-a noului vleat?', 'Dupa te-li vatamara tut, si scoala, incarca avutamea adunata di ani di dzile aclo, di la urfane, s-baga di adara na casa in triyeaua, iu arminea calitori dit patrule parti a loclui, s-mica, s-bea si s-nu plateasca pindara.', 'Duse aestu, avigle-avigle pina neadza-nopti, nu s-vidzu tiva. Durnea iapile tu protlu somnu. Citra te-adoara, s-deade ficiorlu pri mina - s-agirseasca natheama, ca dzitea nis: Maca pina tora nu s-vidzu zulapea, niti de-aoa si-nclo nu va yina.', 'Avde vasilelu ca hililu-museat-de-avinator s-u-ncruna feata, si foc s-aprinde! Pitreate niscinti oamini di la palate, s-lu vatama. Cari oamini - t-ul acata si-l fac filii-filii!', 'Tri Viroana si-ni alag;', 'S-apruche, si-li gri:']

In [11]:
print(len(t1), len(t2))
['- Vai de mine si de mine, se mira asta! Eu credeam ca doar vergeaua mea e ce e!', 'Si-l asteapta feciorul pe Inghite-drumuri, il asteapta - dar asta, nimic! Cade pe ganduri baiatul - ce-o fi cu tovarasul sau? Pe unde o fi taind frunza la caini, de-ntarzie atata, ca doar nu-i trebuia mai mult de un ceas, sa se duca si sa se intoarca!', 'Si iar o-ntinde cocosul inspre palatul craiului. Ii iese-n drum raul, care-l opreste in loc.', '- Daca te cheama, du-te!, zise lupul.', 'De-aceea, o si porni, pe loc, spre casa.', 'Asaa... Nu trec insa trei-patru zile, ca iepele lor - ei erau oameni cuprinsi! - si incep a se rari. Herghelegiul innebunea vazand ca-i piere in fiecare zi cate-o iapa, si nu pricepea ce fiara i le omoara: vreun lup, vreun urs - vreun drac, habar n-avea!', 'Zapusala ajunsese in toiul ei. Si-i zice vizirul mosului, care tacea, de parca era fara grai;', 'Si, suparandu-se, imparateasa n-a mai vrut sa-l vesteasca pe imparat ca, de asta data, are fecior.', 'Imparatul, care avea chef in ziua aceea, nu-i zise nimic sateanului, dar astepta sa vada ce-are de gand sa faca omul asta.', 'FATA MOSULUI CEA ISTEATA', '- Era, parintele draga - pe vremea tineretilor mele! Intr-o zi, cand tocmai se nastea taica-miu, ma aflam la bunici si trageam din ciubuc. Cand, pe cine vad - pe strabunicii mei chiar, care veneau in fuga mare, sa dea de veste: Haide, ca s-a nascut tata-tau! Si ce daca s-a nascut? - intreb eu - ce vreti de la mine? Cum, ce vrem - ca trebuie sa te duci dupa nas, sa vina sa-l boteze! - zic ei, bietii batrani.', 'Feciorul, care bagase de seama cat ii cazuse de drag Frumoasei-lumii, isi opri fata pentru el, n-a vrut s-o mai deie craiului. Numai el stia cate patise, pana s-o castige - si acu s-o lase craiului, pica para malaiata? Nici gand!', 'Voinicu, numai ce-o vede si-o si cunoaste, dupa chip, dupa glas, ca era soru-sa, pentru care nu avea el somn de-atata vreme! Dintr-un salt e langa dansa, o imbratiseaza, o saruta cu drag, plang amandoi in hohote, de-ti sfasie inima: de mai bine de trei ani, sarmanii, nu se mai vazusera, nu si-au mai auzit glasul!', '- Haide, ca va dau un aspru - numai n-o omorati, le zise el.', 'Si asteapta el un ceas, asteapta doua ceasuri, dar Nastratin, nicaieri. Pierzandu-si judele rabdarea, trimite un aprod dupa el, sa-l cheme. Cand ajunse aprodul la Nastratin, il gasi fierband niste grau. I-a spus ca e asteptat la judecatorie si ca sa se grabeasca. Nastratin ii zise ca vine numaidecat, ca uite, graul e gata, fiert, creste apoi ca din apa. Bietul aprod, la asemenea traznai - nu mai stiu cum sa se inchine de mirare!', 'Dupa care, isi vazura de drum, unu p-aci, alalalt pe dincolo.']
['Unde-a fi de gasit nasul? Rastorn pamantul, dupa el, ma uit, cotrobai - nasul, nicaieri! O fi pe la bunul Dumnezeu, ma gandesc eu, daca pe pamant nu-i. Si chiar ca acolo era!', 'Sahul Persiei a poftit, de mult, sa stie cati prosti are in imparatia lui, si-l puse pe un vizir al sau sa-i intocmeasca o lista cu ei. Vizirul intocmi lista, dar trecu in capul ei tocmai pe Sahul! Cand vazu Sahul una ca asta, foc se facu!', 'A fost odata - ce nu s-a mai vazut!', 'Vorbi acum si Dona:', 'Chitu se aseza frumusel, gospodareste, si-i dadu drumul:', 'Zi de zi, se trezeau cu vorba asta pe buze: Vai! Doamne, nu ne dai si noua o fetica?!', 'Afland imparatul ca harapul ii scapa fata si omori si balaurul, nu mai putea de bucurie! Ce sa-i deie, ce bine sa-i faca, - hai sa-i dea pe fiica-sa de nevasta. Tocmai pe gandul harapului! Intr-o saptamana urma sa se faca nunta.', '- De ce sa nu-l iau!', 'iar eu m-am intors de acolo, si va aduceam cate o pereche de ghete, dar ghetele fiind ele rosii, si trecand eu pe langa curtea lui Tosi - se repezira cainii, latosii, si mi le smulsera din maini, fiindca li s-a parut lor c-ar fi niscai ficati...', 'Le dadu si lor un aspru, si lua cu el si mata.', '- Tine tava asta, si furculita - sa te joci cu ele, pana gatesc eu ceva de mancare, jos.', '- Mai, magar nerusinat! - zice craiul, nu-ti ajunse cate imi facusi, si inca ma mai injuri! Prindeti-l, ceru dansul, si sa mi-l ardeti in cuptor, scrum sa-l vad!', 'Si - cucurigu-cucurigu... - iata lupul, iata vulpea, care nu asteptara sa fie chemati de doua ori, si se si repezira la iepe, - hap!, hap!, una dupa alta inghitindu-le, de n-a mai ramas nici picior din toata herghelia.', 'In timp ce-l jeleau asa, poftim ca se lasa in jos o pasare cantatoare, care incepe a zice din pliscul ei micut:', 'Si ii zice lui Aude-tot:', nan]

16 16


In [15]:
sentences = ["Lepurle ansare d-iu nu te-astept", # Iepurele sare de unde nu te aştepţi.
             "Si mveaste tu stranile a cafigiluis-acata s-naura ca na catuse pi ninga usa iu durna amiralu.", # Se îmbracă în straiele cafegiului şi începe să miaune ca o pisică pe lângă uşa unde dormea împăratul.
             "Ca-telo-agudi si ficorlu, measa, ntriba Araplu", # De ce îl lovişi pe băiat, masă, întrebă arapul.
             "Nu ti carteasce varnu, ca va la si para ca escu mine.", # Nu se va lega nimeni de tine, căci o să li se pară că sunt eu.
             "Și s-nu pistipsiri, misura tine diznou!", # Și dacă nu crezi, măsoară tu din nou!
             "Nu mi bati, ca t-aduc s-alti petali io!", # Nu mă bate, că ţi-aduc şi-alte potcoave eu!
            ]
correct_translations = ["Iepurele sare de unde nu te astepti.",
                        "Se imbraca in straiele cafegiului si incepe sa miaune ca o pisica pe langa usa unde dormea imparatul.",
                        "De ce il lovisi pe baiat, masa, intreba arapul.",
                        "Nu se va lega nimeni de tine, caci o sa li se para ca sunt eu.",
                        "Si daca nu crezi, masoara tu din nou!",
                        "Nu ma bate, ca ti-aduc si-alte potcoave eu!"]

In [43]:
model = AutoModelForSeq2SeqLM.from_pretrained('nllb_aromanian')
tokenizer = AutoTokenizer.from_pretrained('nllb_aromanian')
model.cuda()
def translate(
    text, src_lang='ro', tgt_lang='rup', 
    a=32, b=3, max_input_length=1024, num_beams=4, **kwargs
):
    """Turn a text or a list of texts into a list of translations"""
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(
        text, return_tensors='pt', padding=True, truncation=True, 
        max_length=max_input_length
    )
    model.eval() # turn off training mode
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams, **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

# translate("Se imbraca in straiele cafegiului si incepe sa miaune ca o pisica pe langa usa unde dormea imparatul.", 'romanian', 'aromanian')
for i, s in enumerate(sentences):
    print(f"Original: {s}")
    print(f"Translated: {translate(s, 'ro', 'rup')[0]}")
    print(f"Correct translation: {correct_translations[i]}")
    print()

Original: Lepurle ansare d-iu nu te-astept
Translated: rup Lupul nu te supara
Correct translation: Iepurele sare de unde nu te astepti.

Original: Si mveaste tu stranile a cafigiluis-acata s-naura ca na catuse pi ninga usa iu durna amiralu.
Translated: rup Si, pe la urma, cafea turceasca isi chema si dansa ca o mata a adormit pe usa, unde dormea imparatul.
Correct translation: Se imbraca in straiele cafegiului si incepe sa miaune ca o pisica pe langa usa unde dormea imparatul.

Original: Ca-telo-agudi si ficorlu, measa, ntriba Araplu
Translated: rup Il intreba cel ce-o fi si ficorul ce-o fi, ce-o fi, ce-o fi, pe harapul casei.
Correct translation: De ce il lovisi pe baiat, masa, intreba arapul.

Original: Nu ti carteasce varnu, ca va la si para ca escu mine.
Translated: rup Nimic nu-ti da la o parte, ca-i sa iei si tu, parca sunt eu...
Correct translation: Nu se va lega nimeni de tine, caci o sa li se para ca sunt eu.

Original: Și s-nu pistipsiri, misura tine diznou!
Translated: rup S

In [24]:
model = AutoModelForSeq2SeqLM.from_pretrained('nllb_aromanian')
tokenizer = AutoTokenizer.from_pretrained('nllb_aromanian')